# Strategy analysis example

Debugging a strategy can be time-consuming. Freqtrade offers helper functions to visualize raw data.
The following assumes you work with SampleStrategy, data for 5m timeframe from Binance and have downloaded them into the data directory in the default location.

## Setup

In [ ]:
from pathlib import Path
from freqtrade.configuration import Configuration

# Customize these according to your needs.

# Initialize empty configuration object
config = Configuration.from_files(["freqtrade-config.json"])
# Optionally, use existing configuration file
# config = Configuration.from_files(["config.json"])

# Define some constants
config["timeframe"] = "1m"
# Name of the strategy class
config["strategy"] = "PKStrategy"
# Location of the data
data_location = Path(config['user_data_dir'], 'data', 'binance')
# Pair to analyze - Only use one pair here
pair = "BTC/USDT"

In [ ]:
import vectorbt as vbt
import numpy as np
import pandas as pd
from datetime import datetime, timedelta
from numba import njit
import itertools
import talib
import ta

In [ ]:

# config = Configuration.from_files(["freqtrade-config.json"])


In [ ]:
# Load data using values set above
from freqtrade.data.history import load_pair_history

candles = load_pair_history(datadir=data_location,
                            timeframe=config["timeframe"],
                            pair=pair,
                            data_format = "json",
                            # data_format = "hdf5",
                            )

# Confirm success
print("Loaded " + str(len(candles)) + f" rows of data for {pair} from {data_location}")
candles.head()

## Load and run strategy
* Rerun each time the strategy file is changed

In [ ]:
# Load strategy using values set above
from freqtrade.resolvers import StrategyResolver
strategy = StrategyResolver.load_strategy(config)

# Generate buy/sell signals using strategy
df = strategy.analyze_ticker(candles[candles.date >= pd.Timestamp('2021-04-01',).tz_localize('US/Eastern')], {'pair': pair})
df.tail()

### Display the trade details

* Note that using `data.head()` would also work, however most indicators have some "startup" data at the top of the dataframe.
* Some possible problems
    * Columns with NaN values at the end of the dataframe
    * Columns used in `crossed*()` functions with completely different units
* Comparison with full backtest
    * having 200 buy signals as output for one pair from `analyze_ticker()` does not necessarily mean that 200 trades will be made during backtesting.
    * Assuming you use only one condition such as, `df['rsi'] < 30` as buy condition, this will generate multiple "buy" signals for each pair in sequence (until rsi returns > 29). The bot will only buy on the first of these signals (and also only if a trade-slot ("max_open_trades") is still available), or on one of the middle signals, as soon as a "slot" becomes available.  


In [ ]:
# Report results
print(f"Generated {df['buy'].sum()} buy signals")
data = df.set_index('date', drop=False)
data.tail()

## Load existing objects into a Jupyter notebook

The following cells assume that you have already generated data using the cli.  
They will allow you to drill deeper into your results, and perform analysis which otherwise would make the output very difficult to digest due to information overload.

### Load backtest results to pandas dataframe

Analyze a trades dataframe (also used below for plotting)

In [ ]:
from freqtrade.data.btanalysis import load_backtest_data, load_backtest_stats

# if backtest_dir points to a directory, it'll automatically load the last backtest file.
backtest_dir = config["user_data_dir"] / "backtest_results"
# backtest_dir can also point to a specific file 
# backtest_dir = config["user_data_dir"] / "backtest_results/backtest-result-2020-07-01_20-04-22.json"

In [ ]:
# You can get the full backtest statistics by using the following command.
# This contains all information used to generate the backtest result.
stats = load_backtest_stats(backtest_dir)

strategy = 'PKStrategy'
# All statistics are available per strategy, so if `--strategy-list` was used during backtest, this will be reflected here as well.
# Example usages:
print(stats['strategy'][strategy]['results_per_pair'])
# Get pairlist used for this backtest
print(stats['strategy'][strategy]['pairlist'])
# Get market change (average change of all pairs from start to end of the backtest period)
print(stats['strategy'][strategy]['market_change'])
# Maximum drawdown ()
print(stats['strategy'][strategy]['max_drawdown'])
# Maximum drawdown start and end
print(stats['strategy'][strategy]['drawdown_start'])
print(stats['strategy'][strategy]['drawdown_end'])


# Get strategy comparison (only relevant if multiple strategies were compared)
print(stats['strategy_comparison'])


In [ ]:
# Load backtested trades as dataframe
trades = load_backtest_data(backtest_dir)

# Show value-counts per pair
trades.groupby("pair")["sell_reason"].value_counts()

In [ ]:
trades

### Load live trading results into a pandas dataframe

In case you did already some trading and want to analyze your performance

In [ ]:
from freqtrade.data.btanalysis import load_trades_from_db

# Fetch trades from database
trades = load_trades_from_db("sqlite:///tradesv3.sqlite")

# Display results
trades.groupby("pair")["sell_reason"].value_counts()

## Analyze the loaded trades for trade parallelism
This can be useful to find the best `max_open_trades` parameter, when used with backtesting in conjunction with `--disable-max-market-positions`.

`analyze_trade_parallelism()` returns a timeseries dataframe with an "open_trades" column, specifying the number of open trades for each candle.

In [ ]:
from freqtrade.data.btanalysis import analyze_trade_parallelism

# Analyze the above
parallel_trades = analyze_trade_parallelism(trades, '1m')

parallel_trades.plot()

## Plot results

Freqtrade offers interactive plotting capabilities based on plotly.

In [ ]:
from freqtrade.plot.plotting import  generate_candlestick_graph
# Limit graph period to keep plotly quick and reactive

# Filter trades to one pair
trades_red = trades.loc[trades['pair'] == pair]

data_red = data['2021-06-01':'2021-06-5']
# Generate candlestick graph
graph = generate_candlestick_graph(pair=pair,
                                   data=data_red,
                                   trades=trades_red,
                                   indicators1=['sma20', 'ema50', 'ema55'],
                                   indicators2=['rsi', 'macd', 'macdsignal', 'macdhist']
                                  )




In [ ]:
# Show graph inline
# graph.show()

# Render graph in a seperate window
graph.show(renderer="browser")


## Plot average profit per trade as distribution graph

In [ ]:
import plotly.figure_factory as ff

hist_data = [trades.profit_ratio]
group_labels = ['profit_ratio']  # name of the dataset

fig = ff.create_distplot(hist_data, group_labels,bin_size=0.01)
fig.show()


In [1]:
from __future__ import print_function
from ipywidgets import interact, interactive, fixed, interact_manual
# import ipywidgets as widgets
from ipywidgets import *
from IPython.display import display
import IPython
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import numpy as np

from datetime import datetime
import talib
import pandas_ta as ta
from talib.abstract import *
from math import *
import vectorbt as vbt
import json
%matplotlib inline

In [2]:
from pathlib import Path
from freqtrade.configuration import Configuration

# Customize these according to your needs.

# Initialize empty configuration object
config = Configuration.from_files(["freqtrade-config.json"])
# Optionally, use existing configuration file
# config = Configuration.from_files(["config.json"])

# Define some constants
config["timeframe"] = "1m"
# Name of the strategy class
config["strategy"] = "PKStrategy"
# Location of the data
data_location = Path(config['user_data_dir'], 'data', 'binance')
# Pair to analyze - Only use one pair here
pair = "BTC/USDT"
##############################

from freqtrade.data.history import load_pair_history

candles = load_pair_history(datadir=data_location,
                            timeframe=config["timeframe"],
                            pair=pair,
                            data_format = "json",
                            # data_format = "hdf5",
                            )

# Confirm success
print("Loaded " + str(len(candles)) + f" rows of data for {pair} from {data_location}")

Loaded 249118 rows of data for BTC/USDT from freq-user-data/data/binance


In [3]:


##############################
# Load strategy using values set above
from freqtrade.resolvers import StrategyResolver
strategy = StrategyResolver.load_strategy(config)

data = candles.set_index('date', drop=False)
data = data[data.date >= pd.Timestamp('2021-06-01').tz_localize('US/Eastern')]
# Generate buy/sell signals using strategy
# df = strategy.analyze_ticker(data, {'pair': pair})
# df.tail()
# print(f"Generated {ddf['buy'].sum()} buy signals")
# data = df.set_index('date', drop=False)
# data.tail()


Feel free to submit an issue or Pull Request enhancing this document if you would like to share ideas on how to best analyze the data.

In [82]:
debug_view = widgets.Output(layout={'border': '1px solid black'})

strategy = StrategyResolver.load_strategy(config)
# ddf = df.copy()
ddf = data.copy()
dlen = len(ddf)
capital = strategy.config['starting_balance']
sl_n2= IntSlider(description="n2", min=0, max=floor(log(dlen, 2.0)), step=1, value=9)
sl_w = IntSlider(description="w", min=0, max=floor(dlen/pow(2,9)), step=1, value=5)
def update_sl_w_range(*args):
    wsize = pow(2,sl_n2.value)
    sl_w.max = floor(dlen/wsize)*2 -1
sl_n2.observe(update_sl_w_range, 'value')


# slCoolOff = IntSlider(description="cool", min=0, step=5, max=600, value=30)
# slDowntrendWait = IntSlider(description="dnt", min=0, max=100, value=30)

slStopLoss = FloatSlider(description="sl", min=0, step=0.05, max=15,value=1.92, continuous_update=False)
slTrailingStopLoss = FloatSlider(description="ts", min=0, step=0.2, max=15,value=10, continuous_update=False)
slTakeProfit = FloatSlider(description="tp", min=0, step=0.2, max=15, value=10, continuous_update=False)

# slMa1N = IntSlider(description="ma1n", min=0, step=3, max=400, value=50)
# slMa2N = IntSlider(description="ma2n", min=0, step=5, max=600, value=100)
slBBTP = IntSlider(description="bbtp", min=10, step=3, max=200, value=30, continuous_update=False)
slBBUP = FloatSlider(description="bbup", min=0, step=0.05, max=5,value=2, continuous_update=False)
slBBDN = FloatSlider(description="bbdn", min=0, step=0.05, max=5,value=2, continuous_update=False)
slBBMACH = FloatSlider(description="bbch", min=0, step=0.05, max=3,value=0.5, continuous_update=False)

slRSB = IntSlider(description="rsb", min=10, step=1, max=200, value=14, continuous_update=False)


# slStK = IntSlider(description="stk", min=0, step=1, max=300, value=14)
# slStD = IntSlider(description="std", min=0, step=1, max=200, value=3)
# slStSm = IntSlider(description="stsm", min=0, step=1, max=200, value=2)
# slStT = IntSlider(description="stst", min=0, step=1, max=100, value=20)

# slMa1c = FloatSlider(description="ma1c", min=-5, step=0.01, max=5, value=-0.44)
# slMa2c = FloatSlider(description="ma2c", min=-5, step=0.01, max=5, value=0.02)
# slTrN = IntSlider(description="trn", min=0, step=5, max=800, value=160)
# slMrN = IntSlider(description="mrn", min=0, step=5, max=800, value=300)

# slXrr = FloatSlider(description="xrr", min=1, step=0.2, max=20, value=2)

sui = widgets.VBox([
    widgets.HBox([slTrailingStopLoss, slStopLoss, slTakeProfit]), 
    widgets.HBox([slBBTP, slBBUP, slBBDN, slBBMACH])
])

wui = widgets.HBox([sl_n2, sl_w])

trades = pf = wst = wed = wsz = wdf = False
ddf_entries = ddf_profits = ddf_losses = ddf_cap = None
# iKama1 = Interact.@map kama(&ivdf.close, n=&slKama1N, nfast=&slKama1Fast, nslow=&slKama1Slow)
# iKama1ChannelL = Interact.@map &iKama1 * (1-&slKama1Channel/100) 
# cool, dnt,
idx_ddf = np.arange(dlen)
# , stk, std, stsm, stt , ma1n, ma2n, ma1c, ma2c, trn, mrn, xrr,
def backtest(sl, ts, tp, bbtp, bbup, bbdn, bbch):
    global ddf, pf, trades, ddf_entries, ddf_profits, ddf_losses, ddf_cap
    
#     with open(f'./strat-params.json', "w") as f: f.write(json.dumps({"w2log": sl_n2.value, "w":sl_w.value, "sl":slStopLoss.value, "ts": slTrailingStopLoss.value, 
#            "tp": slTakeProfit.value, "ma1n": slMa1N.value, "ma2n": slMa2N.value, "ma1c": slMa1c.value, "ma2c": slMa2c.value,"trn": slTrN.value })) ;
    
#     ddf = strategy.analyze_ticker(data[data.date >= pd.Timestamp('2021-05-01',).tz_localize('US/Eastern')], {'pair': pair})
    
# Generate buy/sell signals using strategy


    # cust_bb_ma_ch = 0.5
    # cust_bb_timeperiod = 20
    # cust_bb_nbdevdnh = 2.
    strategy.stoploss = -sl/100
    strategy.cust_tp_stop = tp
    strategy.cust_ts_stop = ts
    
    strategy.cust_bb_timeperiod = bbtp
#     strategy.cust_params['bb_nbdevup'] = bbup
    strategy.cust_bb_nbdevdnh = bbdn
    strategy.cust_bb_ma_ch = bbch
#     strategy.cust_params['rsb'] = rsb
    
#     ddf = strategy.analyze_ticker(data, {'pair': pair})
    ddf = strategy.populate_indicators(ddf, {'pair': pair})
    ddf = strategy.populate_buy_trend(ddf, {'pair': pair})
#     ddf = strategy.populate_sell_trend(ddf, {'pair': pair})
   
#     vtma1 = vbt.MA.run(ddf['open'], ma1n)

    ddf['buy'] = ddf['buy'].astype(bool).vbt.signals.first()

    ddf['sell'] = vbt.OHLCSTEX.run(
        ddf['buy'], 
        ddf['open'], 
        ddf['high'], 
        ddf['low'], 
        ddf['close'], 
        sl_stop=sl/100,
        tp_stop=tp/100,
        ts_stop=ts/100,
        stop_type=None, hit_price=None
    ).exits

    # global pf;
    pf = vbt.Portfolio.from_signals(ddf['close'], entries=ddf['buy'], 
                                    exits=ddf.sell,
                                    direction='longonly', init_cash=capital, fees=0.001, slippage=0.0025, freq='1m')


    trades = pf.trades.records
    trades['cap'] = trades['pnl'].cumsum() + capital
    ddf_cap = pd.merge(ddf[["date"]], trades[["exit_idx", "cap"]], left_on=idx_ddf, right_index=False, 
    right_on="exit_idx", how="left", sort=False).fillna(method='ffill').set_index('date')

    
    with open(f'./strategy-results/{datetime.now():strat_%Y-%m-%d}.log', "a") as f: 
        f.write(json.dumps(dict({   
            "dt": f'{datetime.now():%Y-%m-%d %HH:%MM:%SS}',
            "period_start": f'{ddf.index[0]}',
            "period_end": f'{ddf.index[-1]}',
            "params": dict({ 'sl': sl, 'ts': ts, 'tp': tp, 'bbtp': bbtp, 'bbdn': bbdn, 'bbup': bbup, 'bbch': bbch}),
            "strategy": "2EMASTH",
            "profit": floor(100* (trades.iloc[-1].cap / capital - 1)) if len(trades.index) else 0. 
        })
    )); f.write("\n");
    
    
    trades_profits = trades[trades['pnl'] > 0]
    trades_losses = trades[trades['pnl'] < 0]    
    
    ddf_entries =  ddf.iloc[trades['entry_idx'].values]
    ddf_profits = ddf.iloc[trades_profits['exit_idx'].values]
    ddf_losses = ddf.iloc[trades_losses['exit_idx'].values]
    
    
    sl_w.value = sl_w.value + 1 if sl_w.value % 2 ==0 else sl_w.value - 1
    
#     display(wui, wout)
    
#     fig, (ax1, ax2) = plt.subplots(1, 2,figsize=(16,4), sharex='col', gridspec_kw={'width_ratios': [3,1]})

    

with open('./strat-params.json') as f: 
    js = json.loads(f.read())
    slStopLoss.value=js["sl"]; slTrailingStopLoss.value=js["ts"]; slTakeProfit.value=js["tp"];
    slBBMACH.value=js["bbch"]; slBBTP.value=js['bbtp']; slBBUP.value=js['bbup']; slBBDN.value=js['bbdn'];

    print('sout done', sl, '\n')
    
def showplt(w2log, w):
    global wst, wed, wsz, wdf, ddf, ddf_entries, ddf_profits, ddf_losses, ddf_cap
    
    if not pf: return
    
    wsz = floor(pow(2,w2log))
    wst = floor(w * wsz / 2)
    wed = floor(wst + wsz)
    
    
    wdf = ddf[200:].iloc[wst:wed,:]
    
    fig = plt.figure(constrained_layout=True,figsize=(16,5))
    gs0 = fig.add_gridspec(1, 2, width_ratios=[3,1])

    gs00 = gs0[0,0].subgridspec(4, 1, height_ratios=[4,1,1,1])
    gs01 = gs0[0,1].subgridspec(2, 1)

    ax1 = fig.add_subplot(gs00[0,0])
    ax2 = fig.add_subplot(gs00[1,0])
    ax5 = fig.add_subplot(gs00[2,0])

    ax3 = fig.add_subplot(gs01[0,0])
    ax4 = fig.add_subplot(gs01[1,0])


#     wma1 = wdf['ma1']; wma2 = ma2[wdf.index]; wma3 = ma3[wdf.index];
    
    ax1.plot(wdf['close'], lw=0.5)
    ax1.plot(wdf.bb_lowerband, lw=0.5)
#     ax1.plot(wdf.bb_ma, lw=0.5)
#     ax1.plot(wdf.bb_upperband, lw=0.5)
    # ax1.plot(wdf['rmin'], lw=0.5)
    # ax1.plot(wdf['rmax'], lw=0.5)
    # ax1.plot(wdf['ma1'], lw=0.5)
    # ax1.plot(wdf['ma2'], lw=0.5)
    
    ax1.fill_between(wdf.index, wdf.bb_ma_lowerband, wdf.bb_ma, alpha=0.15)
    # ax1.fill_between(wdf.index, wdf['ma1'], wdf['ma1c'][wdf.index], alpha=0.15)
    # ax1.fill_between(wdf.index, wdf['ma2'], wdf['ma2c'][wdf.index], alpha=0.15)

    if ddf_entries is not None:
        wddf_entries = ddf_entries[ddf_entries.index.isin(wdf.index)]
        wddf_profits = ddf_profits[ddf_profits.index.isin(wdf.index)]
        wddf_losses = ddf_losses[ddf_losses.index.isin(wdf.index)]

        wddf_trades = ddf_cap[wst:wed]    

        ax1.vlines(wddf_entries.index,wddf_entries['open'] * 0.97,wddf_entries['open'] * 1.02, lw=0.5, color='blue')
        ax1.vlines(wddf_profits.index,wddf_profits['open'] *0.97,wddf_profits['open'] * 1.02, lw=0.5, color='green')
        ax1.vlines(wddf_losses.index,wddf_losses['open'] * 0.97,wddf_losses['open'] * 1.02, lw=0.5, color='red')
    
#     ax2.plot(wdf.htsine, color="blue")
#     ax2.plot(wdf.htleadsine, color="orange")
    # ax2.plot(wdf.xrr,lw=0.5,color='green')

#     ax2.plot(wslowk,lw=0.5,color='orange')
#     ax2.plot(wslowd,lw=0.5,color='teal')
#     ax2.axhline(y=20, color='b', linestyle='--',lw=0.5)
#     ax2.axhline(y=80, color='b', linestyle='--',lw=0.5)

#     ax2.vlines(wddf_entries.index,0,100, lw=0.5, color='blue')
#     ax2.vlines(wddf_profits.index,0,100, lw=0.5, color='green')
#     ax2.vlines(wddf_losses.index,0,100, lw=0.5, color='red')

    if ddf_cap is not None:
        ax3.plot(ddf_cap['cap'])
        ax4.plot(wddf_trades['cap'])
    
#     ax5.plot(wdf['rsi'])
    plt.show()
    print(pf.stats())
    print(pf.returns_stats())


    
sout = widgets.interactive_output(backtest, {"sl":slStopLoss, "ts": slTrailingStopLoss, "tp": slTakeProfit, 
#                                            "ma1n": slMa1N, "ma2n": slMa2N, "ma1c": slMa1c, "ma2c": slMa2c,"trn": slTrN, 'mrn': slMrN, 'xrr': slXrr,
                                           "bbtp": slBBTP, "bbup": slBBUP, "bbdn": slBBDN, "bbch": slBBMACH,
#                                            "rsb": slRSB
                                          })
    
wout = widgets.interactive_output(showplt, {"w2log":sl_n2, "w":sl_w})

display(sui, sout)
display(wui, wout)

@debug_view.capture(clear_output=True)
def sout__(v):
    print(v)
    

Output()

Output()

In [76]:
# sl_w.notify_change({'name': 'w', 'type': 'value'})
# dir(sl_w)
sl_w.value = sl_w.value + 1 if sl_w.value % 2 ==0 else sl_w.value - 1

In [54]:
dir(wout)

['_Output__counter',
 '__class__',
 '__del__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__enter__',
 '__eq__',
 '__exit__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__setstate__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_add_notifiers',
 '_append_stream_output',
 '_call_widget_constructed',
 '_comm_changed',
 '_compare',
 '_cross_validation_lock',
 '_default_keys',
 '_display_callbacks',
 '_dom_classes',
 '_flush',
 '_gen_repr_from_keys',
 '_get_embed_state',
 '_get_trait_default_generator',
 '_handle_custom_msg',
 '_handle_displayed',
 '_handle_msg',
 '_holding_sync',
 '_ipython_display_',
 '_is_numpy',
 '_lock_property',
 '_log_default',
 '_model_id',
 '_model_module',
 '_model_module_version',
 '_model_name',
 '_msg_callbacks',
 '_notify_obs

In [ ]:
# bb = vbt.talib('BBANDS').run(ddf.close, timeperiod=2, nbdevup=2, nbdevdn=2)
close_above_bblow = ddf.bb_lowerband > ddf.close
sig = close_above_bblow & ~(close_above_bblow.shift(1, fill_value=False))
sig[sig]

In [ ]:
# dir(bb)
# ddf['bb_lowerband'].vbt.above(1)
#  above(ddf.close, crossover=True)
len(ddf)

In [ ]:
ddf.buy.vbt.signals.first()